In [26]:
import pandas as pd

# GitHub raw URL for the CSV file
url = "https://raw.githubusercontent.com/CBDRH/hds-datathon-data/refs/heads/main/release/epiwatch-latest.csv"

# Read the CSV file from the URL into a pandas DataFrame
data = pd.read_csv(url)




In [27]:
data.head()

,title,publication-date,event-date,country,location,lat_long,diseases,syndromes,url
0,90123 people sick in one day in the country; D...,2020-09-16,2017-09-16,India,NaN,"23.379, 79.443",covid19,NaN,https://www.manoramaonline.com/news/latest-new...
1,Ludhiana District remains at receiving end;,2020-09-16,2020-09-16,India,"Punjab, Ludhiana, Ludhiana","30.907, 75.849",other,severe acute respiratory syndrome,https://www.tribuneindia.com/news/ludhiana/lud...
2,CIDRAP - India's COVID-19 total tops 5 million...,2020-09-16,2020-09-16,India,NaN,"23.379, 79.443",covid19,NaN,https://flutrackers.com/forum/forum/local-regi...
3,[Myarklamiss.com] LOUISIANA - State reports 42...,2020-09-15,2020-09-15,United States,Louisiana,"30.186, -91.426",covid19,NaN,http://outbreaks.globalincidentmap.com/eventde...
4,[1010 WINS] NEW JERSEY - New Jersey reports 9 ...,2020-09-15,2020-09-15,United States,New Jersey,"39.613, -74.729",covid19,NaN,http://outbreaks.globalincidentmap.com/eventde...


In [28]:
missing_values = data.isnull().sum()
print("\nMissing Values:\n", missing_values)


Missing Values:
 title                   0
publication-date        0
event-date          34106
country                 0
location            17494
lat_long             2320
diseases             2163
syndromes           38691
url                     0
dtype: int64


In [31]:
# Get unique values and their occurrences
value_counts = data['diseases'].value_counts()

# Print unique values and occurrences
print("Unique Values and their Occurrences:")
print(value_counts)

Unique Values and their Occurrences:
diseases
covid19                                           21235
dengue                                             4364
Mpox                                               4101
avian influenza/unspecified                        3356
influenza                                          1109
                                                  ...  
covid19, measles, tuberculosis                        1
dengue, leishmaniasis                                 1
malaria, measles, mumps                               1
dengue, malaria, leptospirosis, Scrub typhus          1
Mpox, respiratory syncytial virus, candidiasis        1
Name: count, Length: 334, dtype: int64


In [21]:
data['event-date'] = data['event-date'].fillna('unknown')
data['location'] = data['location'].fillna('unknown')
# data['lat_long'] = data['lat_long'].fillna('unknown')
data['diseases'] = data['diseases'].fillna('unknown')
data['syndromes'] = data['syndromes'].fillna('unknown')


In [22]:
data = data.drop(columns=['lat_long'])

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43126 entries, 0 to 43125
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             43126 non-null  object
 1   publication-date  43126 non-null  object
 2   event-date        43126 non-null  object
 3   country           43126 non-null  object
 4   location          43126 non-null  object
 5   diseases          43126 non-null  object
 6   syndromes         43126 non-null  object
 7   url               43126 non-null  object
dtypes: object(8)
memory usage: 2.6+ MB


In [24]:
missing_values = data.isnull().sum()
print("\nMissing Values:\n", missing_values)


Missing Values:
 title               0
publication-date    0
event-date          0
country             0
location            0
diseases            0
syndromes           0
url                 0
dtype: int64


In [1]:
import pandas as pd
import openai
import json
import re

# Load the dataset
file_path = "part1.csv"  # Replace with your file path
df = pd.read_csv(file_path)

# Filter out rows with specific values in the 'extracted' column
exclude_values = ["ClientError", "Translation failed", "Unknown", "Access Denied"]
df = df[~df["extracted"].isin(exclude_values)]

# Initialize OpenAI
openai.api_base = "http://localhost:1234/v1"
openai.api_key = "not-needed"

# List of target diseases
target_diseases = {
    'influenza', 'influenza a/h1n1', 'influenza a/h5n8', 'influenza a/h5n1',
    'influenza a/h3n2', 'influenza a/h9n2', 'influenza a/h7n9', 'influenza b',
    'influenza/h5n5', 'influenza a/h5n6', 'covid19', 'mpox', "legionnaires'",
    'dengue', 'measles', 'cholera'
}

# Function to process the extracted diseases and symptoms data
def process_extracted_data(data, target_set=None):
    results, people_affected = [], []
    seen = set()

    for entry in data:
        if isinstance(entry, dict):
            name = entry.get("name", "").strip().lower()
            if (target_set is None or name in target_set) and name not in seen:
                seen.add(name)
                results.append(name)
                people_affected.append(entry.get("people_affected", 0))

    return results, people_affected

# Preprocessing JSON response
def preprocess_response(content):
    try:
        content = re.sub(r"```(json)?", "", content).strip()
        return json.loads(content)
    except json.JSONDecodeError:
        return None

# Function to extract information from text
def extract_information_from_text(text):
    if pd.isna(text):
        return [[], [], [], []]

    try:
        response = openai.ChatCompletion.create(
            model="local-model",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "Extract the following information from the text in strict JSON format: "
                        '{"diseases": [{"name": "disease_name", "people_affected": int}], '
                        '"symptoms": [{"name": "symptom_name", "people_affected": int}]}. '
                        "Ensure JSON format is valid and does not include extra text or explanations."
                    )
                },
                {"role": "user", "content": text},
            ],
            temperature=0.7,
        )
        content = response.choices[0].message.content.strip()
        processed_content = preprocess_response(content)

        if processed_content is None:
            return [[], [], [], []]

        # Process diseases and symptoms
        diseases_data = processed_content.get("diseases", [])
        diseases, disease_people_affected = process_extracted_data(diseases_data, target_diseases)

        symptoms_data = processed_content.get("symptoms", [])
        symptoms, symptom_people_affected = process_extracted_data(symptoms_data)

        return [diseases, disease_people_affected, symptoms, symptom_people_affected]
    except Exception as e:
        print(f"Extraction error: {e}")
        return [[], [], [], []]

# Apply the extraction function to each row
df[
    [
        "diseases1",
        "disease_people_affected",
        "symptoms1",
        "symptom_people_affected",
    ]
] = pd.DataFrame(df["extracted"].apply(extract_information_from_text).tolist(), index=df.index)


# Save the processed data and summaries to new CSV files
output_file_path = "processed_data_cleaned2.csv"


df.to_csv(output_file_path, index=False)


print(f"Processed data saved to {output_file_path}")

Extraction error: Request timed out: HTTPConnectionPool(host='localhost', port=1234): Read timed out. (read timeout=600)
Extraction error: Request timed out: HTTPConnectionPool(host='localhost', port=1234): Read timed out. (read timeout=600)
Extraction error: 'str' object has no attribute 'get'
Extraction error: Request timed out: HTTPConnectionPool(host='localhost', port=1234): Read timed out. (read timeout=600)
Extraction error: 'str' object has no attribute 'get'
Extraction error: 'list' object has no attribute 'get'
Extraction error: Request timed out: HTTPConnectionPool(host='localhost', port=1234): Read timed out. (read timeout=600)
Extraction error: 'str' object has no attribute 'get'
Extraction error: 'list' object has no attribute 'get'
Processed data saved to processed_data_cleaned2.csv
